In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
%matplotlib inline

# Estructura
1. [Introducción al problema](#intro)
2. [Carga de los datos](#cargaDatos)
3. [Comprensión de los datos](#Comprension)
4. [Preparación de los datos](#Preparacion)
5. [Modelado](#Modelo)
6. [Conclusiones](#Conclusion)

## Introducción al problema

La Fundación Bill & Melinda Gates dispone un programa de Servicios Financieros dirigido a las personas más
desfavorecidas del planeta. Con el objetivo de conocer las necesidades de la población, la Fundación ha realizado una
encuesta sobre diferentes aspectos de la población India.
El datasets contiene información demográfica y de comportamiento de una muestra
representativa de encuestados de India y su uso de servicios financieros y móviles tradicionales. Las encuestas han sido
realizadas por InterMedia para ayudar a las personas más pobres del mundo a aprovechar los teléfonos móviles y otras
tecnologías digitales para acceder a herramientas financieras y participar más plenamente en sus economías locales. Las
mujeres en estas comunidades, en particular, a menudo suelen estar excluidas del sistema financiero formal. Al predecir el
género, y los patrones de comportamiento de hombres y mujeres es clave para la extraiga a seguir por los servicios
financieros. Las conclusiones de estas investigaciones pueden influir en los planes de mejora de estas barreras que
disponen las mujeres en las economías en desarrollo y permitirá desarrollar nuevas herramientas financieras que ayude a
las mujeres y a sus familias a salir de la pobreza.

**Objetivo:**
Analizar los datos y construir modelos de machine learning y estadísticos para
predecir el género de cada encuestado.

<a id='cargaDatos'></a>
## Carga de los datos

In [2]:
train = pd.read_csv('train.csv',sep =",")
test = pd.read_csv('test.csv',sep =",")

/home/dsc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12,49,51,56,91,126,145,163,165,166,167,168,169,171,173,174,176,177,242,244,246,248,252,255,274,290,291,292,294,295,296,332,344,366,374,376,397,414,440,491,620,634,639,642,643,645,710,713,760,769,810,829,929,954,979,1001,1002,1003,1004,1005,1024,1037,1041,1043,1062,1086,1099,1100,1121,1129,1136,1152,1153,1166,1168,1182,1193,1204,1205,1207,1208,1216,1226,1228,1230,1232,1234) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/dsc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (11,13,48,50,55,90,125,144,162,163,164,166,168,169,170,171,172,174,176,241,243,245,247,251,254,273,289,291,292,293,295,296,312,313,314,331,343,365,373,375,396,413,439,490,619,623,630,633,638,642,644,669,712,759,768,788,809,828,928,953,978,1000,1001,1002,1003,1004,10

Unimos ambos ficheros para realizar todas las transformaciones sobre ambos a la vez. Al final los separaremos que los identificaremos por el campo is_female

In [3]:
train['id']=train['train_id']
train=train.drop('train_id', axis=1)
test['id']=test['test_id']
test=test.drop('test_id', axis=1)

In [4]:
train.shape

(18255, 1235)

In [5]:
test['is_female']=99

In [6]:
test.shape

(27285, 1235)

In [7]:
df_union=pd.concat([train,test])
df_union.shape

(45540, 1235)

Como la variable objetivo es is_female vamos a ver si todos los registros del df tienen valores validos (0,1,99).

In [8]:
df_union['is_female'].unique()

array([ 1,  0, 99])

<a id='Comprension'></a>
## Comprension de los datos

Debido al tiempo limitado y la imposibilidad de comprobar cada una de las variables para ver por que valor cambiar los nulos en cada caso vamos a eliminar aquellas columnas con gran cantidad de nulos (>10% de los registros). De esta forma eliminamos ruido a la hora de usar un modelo.

In [9]:
df_union.count()

AA14           45540
AA15           45540
AA3            45540
AA4            45540
AA5            14112
AA6            31428
AA7            45540
DG1            45540
DG10b          20596
DG10c          17097
DG11b          16441
DG11c          13475
DG12B_1        15654
DG12B_2        15654
DG12C_1        12822
DG12C_2        12822
DG13_1          8850
DG13_2          8850
DG13_3          8850
DG13_4          8850
DG13_5          8850
DG13_7          8850
DG13_96         8850
DG13_OTHERS       20
DG14            1785
DG14_OTHERS       66
DG3            45540
DG3A           45540
DG3A_OTHERS      137
DG4            45540
               ...  
MT18_96        30391
MT18_OTHERS      157
MT1A           43625
MT2            45540
MT3_1          22401
MT3_2          22401
MT3_3          22401
MT4_1          22401
MT4_2          22401
MT4_3          22401
MT4_4          22401
MT4_5          22401
MT4_6          22401
MT5            22401
MT5_OTHERS        26
MT6            22401
MT6A         

Al ver los valores que toman las columnas vemos que muchas tienen nulos. Todas aquellas que el count nos da menor que 45540. Por lo que vamos a quedarnos con aquellas que tienen más de 41000 valores validos (aproximadamente un 90%).

In [10]:
df_reducido=df_union[df_union.columns[df_union.count()>41000]]

In [11]:
df_reducido.head()

,AA14,AA15,AA3,AA4,AA7,DG1,DG3,DG3A,DG4,DG5_1,...,MMP1_7,MMP1_8,MMP1_9,MMP1_96,MT1,MT10,MT1A,MT2,id,is_female
0,3854,481,3,32,323011,1975,3,4,5,1,...,2,2,2,2,1,2,99.0,2,0,1
1,2441,344,2,26,268131,1981,8,4,5,2,...,2,2,2,2,1,2,4.0,2,1,1
2,754,143,1,16,167581,1995,3,2,2,1,...,2,2,2,2,1,2,2.0,2,2,1
3,5705,604,4,44,445071,1980,3,4,5,1,...,2,2,2,2,2,1,2.0,1,3,1
4,5645,592,4,43,436161,1958,3,4,6,1,...,2,2,2,2,99,2,NaN,2,4,1


Vamos a ver si de las columnas que nos quedan tenemos alguna que siempre tome el mismo valor. En caso de encontrar alguna la eliminamos porque podria distorsionar el estudio.

In [12]:
valores_unicos=[]
for i in df_reducido.columns:
    if len(df_reducido[i].unique()) == 1:
        valores_unicos.append(i)
valores_unicos

['DG5_96', 'MM2_16']

In [13]:
df_reducido2=df_reducido.drop(valores_unicos, axis=1)
df_reducido2.shape

(45540, 260)

In [14]:
df_reducido2.head()

,AA14,AA15,AA3,AA4,AA7,DG1,DG3,DG3A,DG4,DG5_1,...,MMP1_7,MMP1_8,MMP1_9,MMP1_96,MT1,MT10,MT1A,MT2,id,is_female
0,3854,481,3,32,323011,1975,3,4,5,1,...,2,2,2,2,1,2,99.0,2,0,1
1,2441,344,2,26,268131,1981,8,4,5,2,...,2,2,2,2,1,2,4.0,2,1,1
2,754,143,1,16,167581,1995,3,2,2,1,...,2,2,2,2,1,2,2.0,2,2,1
3,5705,604,4,44,445071,1980,3,4,5,1,...,2,2,2,2,2,1,2.0,1,3,1
4,5645,592,4,43,436161,1958,3,4,6,1,...,2,2,2,2,99,2,NaN,2,4,1


Ahora que tenemos menos columnas con nulos vamos a ver cual valor se ajusta mejor (dependiendo del caso podemos usar el 0, el 1, la media, etc.

In [15]:
df_reducido2.columns[df_reducido2.count()<45540]

Index(['DG9a', 'FB19B_1', 'FB19B_2', 'FB19B_3', 'FB19B_4', 'FB19B_5',
       'FB19B_96', 'FB20', 'MM3_1', 'MM3_10', 'MM3_11', 'MM3_12', 'MM3_13',
       'MM3_14', 'MM3_2', 'MM3_3', 'MM3_4', 'MM3_5', 'MM3_6', 'MM3_7', 'MM3_8',
       'MM3_9', 'MT1A'],
      dtype='object')

Vamos a cambiar los NaN por el valor 99 que es el valor que se aplica para el "NS/NC"

In [16]:
df_reducido2.fillna(99,inplace=True)

Comprobamos que ya no hay valores nulos.

In [17]:
len(df_reducido2.columns[df_reducido2.count()<18255])

0

In [18]:
df_reducido2.columns

Index(['AA14', 'AA15', 'AA3', 'AA4', 'AA7', 'DG1', 'DG3', 'DG3A', 'DG4',
       'DG5_1',
       ...
       'MMP1_7', 'MMP1_8', 'MMP1_9', 'MMP1_96', 'MT1', 'MT10', 'MT1A', 'MT2',
       'id', 'is_female'],
      dtype='object', length=260)

<a id='Preparacion'></a>
## Preparacion de los datos

En este punto hemos llegado con 260 columnas entre las que se incluye la variable objetivo (is_female) y el id. Vamos separar el df en dos, por una parte la columna objetivo y el id, y por el otro el resto de columnas. Sobre el segundo df vamos a lanzar un PCA para reducir las columnas a las más representativas, el número de columnas lo vamos a decidir más adelante.

In [19]:
df_objetivo=df_reducido2[['is_female','id']]
df_reducido3=df_reducido2.drop(['is_female','id'], axis=1)

In [20]:
df_objetivo.shape

(45540, 2)

In [21]:
df_reducido3.shape

(45540, 258)

In [22]:
columnas=df_reducido3.columns
columnas

Index(['AA14', 'AA15', 'AA3', 'AA4', 'AA7', 'DG1', 'DG3', 'DG3A', 'DG4',
       'DG5_1',
       ...
       'MMP1_5', 'MMP1_6', 'MMP1_7', 'MMP1_8', 'MMP1_9', 'MMP1_96', 'MT1',
       'MT10', 'MT1A', 'MT2'],
      dtype='object', length=258)

Al revisar las columnas (y con ayuda del diccionario) vemos que hay conjuntos de columnas que vienen de la misma pregunta "padre". En el ejemplo anterior vemos el caso de MMP1. Vamos a agrupar estas variables en una única columna con el fin de reducir columnas de cara a nuestro

In [23]:
variables_agrupadas=[]
for i in df_reducido3.columns:
    variables_agrupadas.append(i.split('_')[0])

In [24]:
variables_agrupadas=list(set(variables_agrupadas))

In [25]:
variables_agrupadas.sort()

In [26]:
variables_agrupadas

['AA14',
 'AA15',
 'AA3',
 'AA4',
 'AA7',
 'DG1',
 'DG3',
 'DG3A',
 'DG4',
 'DG5',
 'DG6',
 'DG8a',
 'DG8b',
 'DG8c',
 'DG9a',
 'DL0',
 'DL1',
 'DL11',
 'DL14',
 'DL15',
 'DL16',
 'DL17',
 'DL18',
 'DL19',
 'DL20',
 'DL21',
 'DL22',
 'DL23',
 'DL24',
 'DL25',
 'DL26',
 'DL4',
 'DL6',
 'FB1',
 'FB13',
 'FB16',
 'FB18',
 'FB19',
 'FB19A',
 'FB19B',
 'FB2',
 'FB20',
 'FB22',
 'FB27',
 'FB29',
 'FB3',
 'FF1',
 'FL1',
 'FL10',
 'FL11',
 'FL12',
 'FL13',
 'FL14',
 'FL15',
 'FL16',
 'FL17',
 'FL18',
 'FL4',
 'FL6',
 'FL7',
 'FL8',
 'FL9A',
 'GN2',
 'GN3',
 'GN4',
 'GN5',
 'IFI1',
 'IFI14',
 'IFI15',
 'IFI18',
 'IFI3',
 'LN1A',
 'LN1B',
 'LN2',
 'MM1',
 'MM2',
 'MM3',
 'MMP1',
 'MT1',
 'MT10',
 'MT1A',
 'MT2']

In [27]:
a=0
for i in variables_agrupadas:
    df_reducido3['C'+str(a)]=df_reducido3[df_reducido3.columns[df_reducido3.columns.str.contains(i)]].mode(axis=1)[0]
    a=a+1
df_reducido3=df_reducido3.drop(columnas, axis=1)
df_reducido3.columns=variables_agrupadas

In [28]:
df_reducido3.shape

(45540, 82)

Calculamos la matriz de covarianza y sus autovalores y autovectores. Creamos las parejas de autovector y autovalor y las ordenamos. Esto lo hacemos para ver con cuantas variables podemos explicar gran parte de los datos.

In [29]:
X_std = StandardScaler().fit_transform(df_reducido3)

In [30]:
matriz_covarianza = np.cov(X_std.T)

autovalor, autovector = np.linalg.eig(matriz_covarianza)

parejas = [(np.abs(autovalor[i]), autovector[:,i]) for i in range(len(autovalor))]

# Ordenamos estas parejas den orden descendiente con la función sort
parejas.sort(key=lambda x: x[0], reverse=True)

In [31]:
total = sum(autovalor)
var_exp = [(i / total)*100 for i in sorted(autovalor, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [32]:
print(cum_var_exp[39])
print(cum_var_exp[49])
print(cum_var_exp[59])
print(cum_var_exp[69])

80.15249375115819
88.02806349211689
94.03059405771342
98.13040845525595


Con esto vemos que con 40 variables podemos explicar el 80%, con 50 variables un 88%, con 60 variables un 94% y con 70 un 98%. Con esto valoramos que nospodemos quedar con 50 variables. Por lo que lanzamos el PCA.

In [33]:
df_reducido3.shape

(45540, 82)

In [34]:
pca = PCA(n_components=40)
principalComponents = pca.fit_transform(X_std)
principalDf = pd.DataFrame(data = principalComponents)

In [35]:
principalDf.shape

(45540, 40)

Ahora vamos a unir este DataFrame con el DataFrame con la columna objetivo

In [36]:
df_objetivo.index=list(range(45540))
principalDf.index=list(range(45540))

In [37]:
finalDf = pd.concat([df_objetivo, principalDf], axis = 1)
finalDf.shape

(45540, 42)

In [38]:
finalDf.head()

,is_female,id,0,1,2,3,4,5,6,7,...,30,31,32,33,34,35,36,37,38,39
0,1,0,9.737250,3.303965,0.142250,0.687251,0.097242,1.064739,-3.719520,-4.462994,...,-1.601129,-0.469435,0.521478,-0.358164,-0.219669,-0.230317,-0.598610,-0.506740,-0.911893,-1.242499
1,1,1,-0.972653,-1.699025,-1.689912,0.005543,1.409236,-1.592054,0.805657,0.016641,...,-0.417720,-0.182292,-0.148753,0.091618,0.096731,-0.213774,0.142963,-0.403848,0.564504,0.723742
2,1,2,-1.016790,-2.960426,-0.543204,1.665542,0.901738,-0.920278,1.639720,-0.810444,...,0.910673,-0.354638,-0.184876,-0.384688,0.521770,-0.690594,-1.565819,0.210840,0.401872,-0.279608
3,1,3,2.546364,1.709745,2.961189,-2.113839,0.443589,0.769548,-1.941849,-1.013783,...,0.508273,-0.519787,0.119159,-0.387567,-1.024621,-0.301853,-0.790689,-0.587380,-0.105933,-0.322942
4,1,4,2.923889,7.963088,-2.300312,3.225992,-4.420551,-3.735542,5.407199,-0.985924,...,0.248119,-0.535884,-0.067643,0.033022,-1.339137,1.865714,1.233480,1.558086,0.588988,-0.722752


<a id='Modelo'></a>
## Modelo

De cara al modelo que vayamos a usar, vamos a separar este df en dos, la parte correspondiente a train y a test.

In [39]:
final_train=finalDf[finalDf['is_female']!=99]
final_train.index=final_train['id']
final_target=final_train['is_female']
final_train=final_train.drop(['id','is_female'], axis=1)
final_train.shape

(18255, 40)

In [40]:
final_test=finalDf[finalDf['is_female']==99]
final_test.index=final_test['id']
final_test=final_test.drop(['id','is_female'], axis=1)
final_test.shape

(27285, 40)

Vamos a empezar usando un Random Forest  ya que podemos meter muchas más variables que con otros modelos más simples, como una regresión lineal.

In [41]:
# CONVIERTE EN NUMPY.MATRIX. Para mejor rendimiento
# -----------------------------------------------------------------------------------------------
data_train = np.matrix(final_train)
data_test  = np.matrix(final_test) 


# MODELO
#---------------------------------------------------------------------------------------------
modelo = RandomForestClassifier(random_state      = 1, n_estimators      = 666, min_samples_split = 2, 
                                min_samples_leaf  = 1, n_jobs            = 1)
modelo.fit(X = final_train, y = final_target)


# PREDICCION
#---------------------------------------------------------------------------------------------
prediccion = modelo.predict(final_test)
prediccion

array([1, 0, 1, ..., 0, 0, 0])

In [42]:
len(prediccion)

27285

In [43]:
final_prediccion = pd.DataFrame({
 'Id':final_test.index, 
 'is_female':prediccion
 })

In [44]:
final_prediccion.head()

,Id,is_female
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1


In [45]:
final_prediccion.to_csv('final_prediccion.csv',index=False)

<a id='Conclusion'></a>
## Conclusiones

### Primera validación:
Una 'validación' muy sencilla que podemos hacer es comparar la proporción hombres/mujeres que tenemos en los datos de train vs esa misma proporción en los resultados que hemos obtenido

In [57]:
train.groupby('is_female').count()['AA3']

is_female
0    8450
1    9805
Name: AA3, dtype: int64

In [60]:
8450/(8450+9805)

0.4628868803067653

In [59]:
final_prediccion.groupby('is_female').count()

,Id
is_female,
0,11920
1,15365


In [61]:
11920/(11920+15365)

0.43687007513285686

Vemos que tiene una proporcion similar.

### Segunda validación:
Para comprobar la robustez del modelo vamos a probarlo con los datos de Train, para ello vamos a separarlo en dos df y lanzar el modelo.

In [77]:
# TRAIN y TEST
#---------------------------------------------------------------------------------------------
marca     = np.random.rand(len(final_train)) < 0.8
train_rob  = final_train.loc[marca,final_train.columns].as_matrix()
test_rob   = final_train.loc[~marca,final_train.columns].as_matrix()
target_train = final_target.loc[marca].as_matrix()
target_test  = final_target.loc[~marca].as_matrix()

# CONVIERTE EN NUMPY.MATRIX. Para mejor rendimiento
# -----------------------------------------------------------------------------------------------
train_rob = np.matrix(train_rob)
test_rob  = np.matrix(test_rob) 


# MODELO
#---------------------------------------------------------------------------------------------
modelo = RandomForestClassifier(random_state      = 1, n_estimators      = 666, min_samples_split = 2, 
                                min_samples_leaf  = 1, n_jobs            = 1)
modelo.fit(X = train_rob, y = target_train)


# PREDICCION
#---------------------------------------------------------------------------------------------
prediccion_rob = modelo.predict(test_rob)
prediccion_rob

array([0, 1, 0, ..., 0, 0, 0])

In [82]:
# METRICAS
#---------------------------------------------------------------------------------------------
print(metrics.classification_report(y_true=target_test, y_pred=prediccion_rob))
print('*******************************************************')
print(pd.crosstab(target_test, prediccion_rob, rownames=['REAL'], colnames=['PREDICCION']))


             precision    recall  f1-score   support

          0       0.81      0.73      0.77      1675
          1       0.78      0.85      0.82      1914

avg / total       0.80      0.79      0.79      3589

*******************************************************
PREDICCION     0     1
REAL                  
0           1221   454
1            283  1631


Como podemos ver en la matriz de confusión y en las metricas que hemos obtenido del modelo vemos que tiene una precision del 80%.